# Linear Regression Examples

This Jupyter notebook presents examples building on theory from the lectures.

In [ ]:
# Boilerplate initialisation code
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 24
plt.rcParams['figure.figsize'] = (20.0, 10.0)

### Example: Wind tunnel data

In a sequence of 8 experiments, a sensor is suspended in a wind tunnel to measure the force $F$ experienced (in Newtons) at various wind speeds $v$ (in m/s). The data is stored in the file `wind_tunnel.csv`.

In [ ]:
data = pd.read_csv('data/wind_tunnel.csv')  # Load data as usual using Pandas
data.head()

Generate a quick plot...

In [ ]:
n = len(data) # number of samples
data.plot.scatter(x='speed', y='force', marker='x')
plt.show()

Extract Numpy arrays from DataFrame for target $y$ and matrix $\mathbf{X}$...

In [ ]:
n = len(data) # number of samples
y = data.force.values.reshape((n,1))
X = np.hstack((np.ones((n,1), dtype=np.float64), data.speed.values.reshape((n,1))))
print(X) # Preview array X; padded with column of ones as required
print(y) # Preview target vector y

With $\mathbf{X}$ and $y$ as matrices, can solve linear system several ways:

In [ ]:
np.linalg.inv(X.T.dot(X)).dot(X.T.dot(y)) # one method

In [ ]:
# alternative way to compute same thing
w_LS = np.linalg.solve(np.dot(X.T, X), np.dot(X.T, y))
print(w_LS)

Produce a 2D plot showing original data and the least-squares line of best fit

In [ ]:
vv = np.linspace(0,80)        # Generate a grid of speed values
FF = w_LS[0,0] + w_LS[1,0]*vv # Generate the corresponding force values 
plt.plot(vv, FF, 'r', X[:,1], y, 'o', markersize=7.5)
plt.xlabel('$v$ [m/s]')
plt.ylabel('$F$ [N]')
plt.grid(True)
plt.show()

To get a sense of how well the line fits the data, compute the *RMSE* (*Root Mean Square Error*) and the *$R^2$ statistic*.

* The RMSE is square root of the average residual squared error (i.e., find the total residual squared error divided by the number of samples and then compute the square root). In terms of the notation used so far, this is $$\displaystyle{\sqrt{\frac{1}{n}\left\|y -\mathbf{X}w_{LS}\right\|^2} = \sqrt{\frac{1}{n}\sum_{k=1}^{n}\left(y_{k} - x_{k}^{T}w_{LS}\right)^{2}}}.$$
* The [$R^2$ statistic (or coefficient of determination)](https://en.wikipedia.org/wiki/Coefficient_of_determination) is the fraction remaining when the ratio of the sum of the squared residuals to the total sum of squares is subtracted from one. The result is a number between 0 and 1 that quantifies how well the regression line explains the data. In effect, this is $$\displaystyle{1 - \left[\sum_{k=1}^{n}\left(y_{k}-\overline{y}\right)^{2}\right]^{-1}} \sum_{\ell=1}^{n}\left(y_{\ell} - x_{\ell}^{T}w_{LS}\right)^{2}.$$

Here, accumulate the sum of squared residuals in `ss_r` and the sum of total squares (differences from mean of $y$) in `ss_t`. These can be used to compute the RMSE and $R^2$.

In [ ]:
ss_t = 0.0 # for accumulating the total sum of squares (i.e., differences of y[k] from y.mean() squared)
ss_r = 0.0 # for accumulating the sum of squared residuals
mean_y = y.mean() # compute once

b0 = w_LS[0,0]
b1 = w_LS[1,0]
for k in range(n):
    y_pred = b0 + b1*X[k,1]
    ss_r += (y_pred - y[k,0]) ** 2
    ss_t += (y[k,0] - mean_y) ** 2

rmse = np.sqrt(ss_r/n)
r2 = 1 - (ss_r/ss_t)
print('RMSE = {}'.format(rmse))
print('R-squared: {}'.format(r2))

### Example: Pearson's height data

These are the heights of 1,078 fathers and their sons (in inches) based on a [famous experiment by Karl Pearson](http://www.randomservices.org/random/data/Pearson.html) around 1903. Random noise was added to the original data, to produce heights to the nearest 0.1 inch. The data is stored in the file `Pearson.csv`.

| Father  |  Son |
| ---  | --- |
| 65.0 | 59.8 |
| 63.3 | 63.2 |
| 65.0 | 63.3 |
| 65.8 | 62.8 |
| 61.1 | 64.3 |
| 63.0 | 64.2 |
| 65.4 | 64.1 |
| $\vdots$ | $\vdots$ |

In [ ]:
# First, read the data into a Pandas DataFrame
data = pd.read_csv('Pearson.csv')
data.head()

In [ ]:
# In this instance, a scatter plot shows the spread of the data
data.plot.scatter(x='Father', y='Son')
plt.show()

Again, set up the vector $y$ and matrix $\mathbf{X}$ as required to compute the weights $w_{LS}$:

In [ ]:
n = len(data)
y = data.Son.values.reshape((n,1)) # Extract as a numpy array
X = np.hstack( (np.ones((n, 1), dtype=np.float64), data.Father.values.reshape((n,1))))

In [ ]:
# Finally, "solve" the required overdetermined system of linear equations X w = y.
w_LS = np.linalg.solve(np.dot(X.T, X), np.dot(X.T, y))
print(w_LS)

In [ ]:
# Produce a 2D plot showing the data and the least-squares line of best fit
FF = np.linspace(data.Father.min(), data.Father.max())
SS = w_LS[0,0] + w_LS[1,0]*FF # Generate 
plt.plot(X[:,1], y, 'o', FF, SS, 'r', linewidth=2)
plt.xlabel('Father heights [in]', fontsize=18)
plt.ylabel('Son heights [in]', fontsize=18)
plt.grid(True)
plt.show()

As before, compute RMSE & $R^2$

In [ ]:
ss_t = 0.0 # for accumulating the total sum of squares (i.e., differences of y[k] from y.mean() squared)
ss_r = 0.0 # for accumulating the sum of squared residuals
mean_y = y.mean() # compute once

b0 = w_LS[0,0]
b1 = w_LS[1,0]
for k in range(n):
    y_pred = b0 + b1*X[k,1]
    ss_r += (y_pred - y[k,0]) ** 2
    ss_t += (y[k,0] - mean_y) ** 2

rmse = np.sqrt(ss_r/n)
r2 = 1 - (ss_r/ss_t)
print('RMSE = {}'.format(rmse))
print('R-squared: {}'.format(r2))

Using Numpy's linear algebra utilities, above loops can be eliminated:

In [ ]:
y_pred = np.dot(X, w_LS)
ss_r = np.linalg.norm(y_pred - y) ** 2
rmse = np.sqrt(ss_r/n)
print('RMSE_linalg = {}'.format(rmse)) # Same as above

In [ ]:
ss_t = np.linalg.norm(y - y.mean()) ** 2
r2 = 1-(ss_r/ss_t)
print('R-squared_linalg = {}'.format(r2)) # Same as above (mostly)

Notice that the $R^2$ statistic is dimensionless while the RMSE is not. The former quantifies the appropriateness of the line fit in a relative sense, whereas the RMSE gives an absolute measure of misfit (which, depending on the scaling of the problem, can be deceptive).

### Example: NHANES data

The table below give the results of a random sample of size 100 from the [2005–2006 National Health and Nutrition Examination Survey (NHANES)](http://www.randomservices.org/random/data/NHANES.html). The variables are Gender (male or female), Age (in years), Weight (in lbs), Height, Leg length, Waist circumference, & Thigh circumference (all in inches). The data is stored in the file `NHANES.csv`.

With this data, we can apply linear regression with multiple independent (or exogenous) variables; this is sometimes called *multiple linear regression*.



In [ ]:
data = pd.read_csv('NHANES.csv')
data.head()

Use Pandas/Numpy techniques for filtering to separate genders:

In [ ]:
women = (data.Gender=='F') # Define boolean series to select each gender from data
men = ~women
women = data.loc[women]
men = data.loc[men]
women.head()

In [ ]:
# Examine a 2D cross-section of the data stratified by gender
plt.scatter(men.Height, men.Weight, color='r', marker='o', linewidth=5, label='Men')
plt.scatter(women.Height, women.Weight, color='b', marker='>', linewidth=5, label='Women')
plt.grid(True)
plt.legend(loc='upper left')
plt.xlabel('Height [in]')
plt.ylabel('Weight [lb]')
plt.show()

Let's think of Weight as a dependent variable and construct a linear regression model using the other variables (except for gender; we've separated out the two categories of subjects) as independent variables. To do this, we'll subselect a sequence of columns from the DataFrame and use those to construct a regression.

In [ ]:
n_men = len(men)
cols = men.columns.symmetric_difference(['Gender', 'Weight'])
men.loc[:,cols].head()

In [ ]:
# Extract target y and matrix X
y_men = men['Weight'].values.reshape((n_men,1))
X_men = np.concatenate((np.ones((n_men,1),dtype=np.float64), men.loc[:, cols].values), axis=1)

Ready to solve $w_{LS} = (\mathbf{X}^{T}\mathbf{X})^{-1}\mathbf{X}^{T}y$ as usual...

In [ ]:
# Determine the regression weights
w_men_LS = np.linalg.solve(np.matmul(X_men.T, X_men), np.dot(X_men.T, y_men))
print(w_men_LS)

In [ ]:
y_pred = X_men.dot(w_men_LS)
ss_r = np.linalg.norm(y_pred - y_men) ** 2
ss_t = np.linalg.norm(y - y.mean()) ** 2
rmse = np.sqrt(ss_r/n_men)
print('RMSE_linalg = {}'.format(rmse))
r2 = 1-(ss_r/ss_t)
print('R-squared_linalg = {}'.format(r2))

### Example: Polynomial regression

As discussed in the lectures, linear regression is *linear* due to the linear dependence on the coefficients. It is possible to have determine coefficients of linear combinations of arbitrary nonlinear functions to fit data. This is appropriate to describe trends (e.g., periodicity) that cannot be described well by degree one polynomials.

As an example, let's try to fit the following points on a rectangular pulse using polynomial functions.

First, generate the data points for the rectangular pulse...

In [ ]:
x = np.linspace(-5, 5, 11).reshape((11,1))
y = np.zeros((11,1))
y[3:6] = 1

In [ ]:
plt.plot(x, y, 'rx', markersize=7.5)
plt.axis('equal')
plt.ylim([-5.5, 5.5])
plt.xlim([-2, 2])
plt.grid(True)
plt.show()

In [ ]:
n = len(x)
X = np.hstack([x**k for k in range(3)])

w2 = np.linalg.solve(np.dot(X.T, X), np.dot(X.T, y))
print(w2)
xx = np.linspace(-5.5, 5.5, 200).reshape((200,1))
y2 = np.hstack((np.ones((200,1)), xx, xx**2)).dot(w2) # Evaluate on grid

plt.plot(x, y, 'rx', markersize=7.5, label='data')
plt.plot(xx, y2, 'b-', linewidth=3, label='deg. 2')
plt.axis('equal')
plt.legend(loc='lower right')
plt.xlim([-5.5, 5.5])
plt.ylim([-2, 2])
plt.grid(True)
plt.show()

In [ ]:
X = np.hstack([x**k for k in range(6)])
w5 = np.linalg.solve(np.dot(X.T, X), np.dot(X.T, y))
print(w5)
y5 = np.hstack((np.ones((200,1)), xx, xx**2, xx**3, xx**4, xx**5)).dot(w5) # Evaluate on grid

plt.plot(x, y, 'rx', markersize=7.5, label='data')
plt.plot(xx, y2, 'b-', linewidth=3, label='deg. 2')
plt.plot(xx, y5, 'm.-', linewidth=3, label='deg. 5')
plt.axis('equal')
plt.legend(loc='lower right')
plt.xlim([-5.5, 5.5])
plt.ylim([-2, 2])
plt.grid(True)
plt.show()

In [ ]:
X = np.hstack([x**k for k in range(10)])
w9 = np.linalg.solve(np.dot(X.T, X), np.dot(X.T, y))
print(w9)
y9 = np.hstack((np.ones((200,1)), xx, xx**2, xx**3, xx**4, 
                xx**5, xx**6, xx**7, xx**8, xx**9)).dot(w9) # Evaluate on grid

plt.plot(x, y, 'rx', markersize=7.5, label='data')
plt.plot(xx, y2, 'b-', linewidth=3, label='deg. 2')
plt.plot(xx, y5, 'm.-', linewidth=3, label='deg. 5')
plt.plot(xx, y9, 'k--', linewidth=3, label='deg. 9')
plt.axis('equal')
plt.legend(loc='lower right')
plt.xlim([-5.5, 5.5])
plt.ylim([-2, 2])
plt.grid(True)
plt.show()

In [ ]:
w2_polyfit = np.polyfit(x.squeeze(), y.squeeze(), 2) # Find coefficients of degree 2 polynomial regression
print(w2.squeeze(), w2_polyfit) # same coefficients, reverse order
y2 = np.polyval(w2_polyfit,xx)